# Bases de datos vectoriales

## Codificar datos a embeddings

In [ ]:
!pip install -U sentence-transformers
!pip install chromadb
!pip install openai

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [ ]:
df = pd.read_csv('/content/imdb_top_1000.csv')

In [ ]:
df.head()

In [ ]:
df['text'] = df.apply(lambda x : x['Overview']+' '+x['Director']+' '+x['Star1']+' '+x['Star2']+' '+x['Star3']+' '+x['Star4'], axis=1)

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
embeddings = model.encode(df['text'],batch_size=64,show_progress_bar=True)

In [ ]:
df['embeddings'] = embeddings.tolist()

In [ ]:
df['ids'] = df.index
df['ids'] = df['ids'].astype('str')

In [ ]:
df

## Chroma

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

In [ ]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key='xxxxxxxx',
    model_name = 'text-embedding-ada-002'
)

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name = 'all-MiniLM-L6-v2')

In [ ]:
chroma_client = chromadb.Client()
client_persistent = chromadb.PersistentClient(path='/content/data_embeddings')

In [ ]:
db = client_persistent.create_collection(name='movies_db', embedding_function=sentence_transformer_ef)

In [ ]:
db.add(
    ids=df['ids'].tolist(),
    embeddings=df['embeddings'].tolist(),
    metadatas= df.drop(['ids','embeddings','text'],axis=1).to_dict('records')
)

In [ ]:
db.peek(1)

## Chroma embeddings

In [ ]:
db_no_embeddings = client_persistent.create_collection(name='movies_db_no_embeddings', embedding_function= sentence_transformer_ef)

In [ ]:
db_no_embeddings.add(
    ids=df['ids'].tolist(),
    documents=df['text'].tolist(),
    metadatas= df.drop(['ids','embeddings','text'],axis=1).to_dict('records')
)

In [ ]:
db_no_embeddings.peek(1)

In [ ]:
db_no_embeddings.delete(ids=['0'])

In [ ]:
db_no_embeddings.peek(1)

### Chroma Query

In [ ]:
results = db_no_embeddings.query(
    query_texts=['a history with elves and a ring'],
    n_results=2
)

In [ ]:
results['metadatas'][0][1]

### Where
1. Estructura

`
{
    "metadata_field": {
        <Operator>: <Value>
    }
}
`
2. Operadores

$eq - equal to (string, int, float)

$ne - not equal to (string, int, float)

$gt - greater than (int, float)

$gte - greater than or equal to (int, float)

$lt - less than (int, float)

$lte - less than or equal to (int, float)

In [ ]:
results_where = db_no_embeddings.query(
    query_texts=['a history with elves and a ring'],
    n_results=2,
    where = {
        'IMDB_Rating' : {
            "$gte":8
        }
    }
)


In [ ]:
results_where

In [ ]:
results_where = db_no_embeddings.query(
    query_texts=['a history with elves and a ring'],
    n_results=2,
    where ={ "$and":[ {
        'IMDB_Rating' : {
            "$gte":8
        }
    },
                      {
        'Released_Year' : {
            "$ne":'2001'
        }
    }] }
)



In [ ]:
results_where

### Cargar índice de Chroma previamente creado

In [ ]:

client_persistent_2 = chromadb.PersistentClient(path="/content/data_embeddings")

In [ ]:
db_2 = client_persistent_2.get_collection('movies_db_no_embeddigs')

In [ ]:
db_2.peek(1)

# Pinecone

In [ ]:
!pip install pinecone-client

In [ ]:
import pinecone
from getpass import getpass

In [ ]:
pincone_api = getpass('Enter the secret value: ')

In [ ]:
pinecone.init(api_key=pincone_api, environment="us-west4-gcp-free")

In [ ]:
dimension_embeddings = len(df['embeddings'][0])
dimension_embeddings

In [ ]:
pinecone.create_index('movies-emebeddings',dimension=dimension_embeddings, metric='cosine')

In [ ]:
index = pinecone.Index('movies-emebeddings')

In [ ]:
df = df.fillna(' ')

In [ ]:
df

In [ ]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size=64

for i in tqdm(range(0, len(df), batch_size)):

    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df[i:i_end]
    # generate embeddings for batch
    ids = batch['ids']
    emb = batch['embeddings']
    metadata = batch.drop(['ids','embeddings','text'],axis=1).to_dict('records')

    # add all to upsert list
    to_upsert = list(zip(ids, emb, metadata))
    # upsert/insert these records to pinecone
    _ = index.upsert(to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

In [ ]:
index.fetch(['0'])

In [ ]:
index.delete(ids=['0'])

In [ ]:
index.fetch(['0'])

### Pinecone query

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
query = 'a history of time travel'
query_vector = model.encode(query).tolist()

responses = index.query(vector=query_vector, top_k = 3, include_metadata=True)

In [ ]:
responses

### Filter

`
filter={
        "genre": {"$eq": "documentary"},
        "year": 2019
    }
`

The metadata filters can be combined with AND and OR:

$eq - Equal to (number, string, boolean)

$ne - Not equal to (number, string, boolean)

$gt - Greater than (number)

$gte - Greater than or equal to (number)

$lt - Less than (number)

$lte - Less than or equal to (number)

$in - In array (string or number)

$nin - Not in array (string or number)


In [ ]:
query = 'a history of time travel'
query_vector = model.encode(query).tolist()

responses = index.query(vector=query_vector, top_k = 3, include_metadata=True,
                        filter={
                            "Genre":{'$ne':"Animation, Adventure, Comedy"}
                        })

In [ ]:
responses

### Load Index

In [ ]:
pinecone.init(api_key=pincone_api, environment="us-west4-gcp-free")

In [ ]:
index_2 = pinecone.Index('movies-emebeddings')

In [ ]:
query = 'a history of an space journey'
query_vector = model.encode(query).tolist()

responses = index_2.query(
  vector=query_vector,
  top_k=3,
  include_metadata=True,
)


In [ ]:
responses